In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [2]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=10000
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=50
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [3]:
'''Preparing dataset'''

df = pd.read_json(os.getcwd()+'/datasets/subtaskA_train_monolingual.jsonl', lines=True)
df = df[['text', 'label']]

df=df.sample(round(SAMPLES_TO_TRAIN))
# test_train_df=df.sample(round(SAMPLES_TO_TRAIN*.2))

# df = pd.read_json(os.getcwd()+'/datasets/subtaskA_dev_monolingual.jsonl', lines=True)
# df = df[['text', 'label']]

# val_df= df.sample(round(SAMPLES_TO_TRAIN*.2))
# test_dev_df= df.sample(round(SAMPLES_TO_TRAIN*.2))

# we balance the training set
print(f'Dataset size before balancing: {df.shape}')
counts = df['label'].value_counts()
sampler = RandomUnderSampler(random_state=42)
x_text, y = sampler.fit_resample(df[['text']], df['label'])

print(f'Dataset size after balancing: {x_text.shape}')
print(f'Entried dropped: {df.shape[0]-x_text.shape[0]}')

# Create a new balanced DataFrame
df = pd.DataFrame({'text': x_text['text'], 'label': y})

# Print the balanced DataFrame
print("\nBalanced DataFrame:")
print(df['label'].value_counts())

Dataset size before balancing: (10000, 2)
Dataset size after balancing: (9382, 1)
Entried dropped: 618

Balanced DataFrame:
label
0    4691
1    4691
Name: count, dtype: int64


In [4]:
'''loading glove'''
embeddings_index={}
with open('../0 playground and indoor/OtherData/glove.6B.200d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embeddings_index[word]=vectors
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [5]:
'''glove building'''

from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def sent2vec(s):
    """ Function Creates a normalized vector for the whole sentence"""
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

print('Training df:')
df_x = np.array([sent2vec(x) for x in tqdm(df['text'])])
print(df_x.shape)
train_y=df['label']


Training df:


100%|██████████| 9382/9382 [00:14<00:00, 665.06it/s]

(9382, 200)


In [6]:
'''Preparing for training'''

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Initialize the StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training data and transform the data
train_x = scaler.fit_transform(df_x)

import pickle

# Save the trained scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


In [7]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [8]:
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.X = torch.from_numpy(X_train.astype(np.float32))
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return {'input_ids': self.X[index], 'labels': self.y[index]}

    def __len__(self):
        return self.len

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
traindata = Data(X_train, y_train)
testdata = Data(X_test, y_test)

In [20]:
# number of features (len of X cols)
input_dim = train_x.shape[-1]

# number of classes (unique of y)
output_dim = 2

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        
        self.lstm1 = nn.LSTM(input_dim, 512, batch_first=True)
        self.ln1 = nn.LayerNorm(512)
        self.dropout1 = nn.Dropout(0.2)
        
        self.lstm2 = nn.LSTM(512, 512, batch_first=True)
        self.ln2 = nn.LayerNorm(512)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc = nn.Linear(512, output_dim)
        
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, labels=None):
        # print(f"Input shape: {input_ids.shape}")
        
        x, _ = self.lstm1(input_ids)
        # print(f"After LSTM1: {x.shape}")

        x = self.ln1(x)
        x = self.dropout1(x)
        
        # print(f"Before LSTM2: {x.shape}")
        
        x, _ = self.lstm2(x)
        # print(f"After LSTM2: {x.shape}")

        x = self.ln2(x)
        x = self.dropout2(x)
        
        x = self.fc(x)
        # print(f"Output shape: {x.shape}")
        
        outputs = (x,)
        if labels is not None:
            loss = self.loss(x, labels)
            outputs = (loss,) + outputs
            
        return (outputs if len(outputs) > 1 else outputs[0])

# Instantiate the model with appropriate dimensions
model = RNNModel()

In [21]:
from transformers import EarlyStoppingCallback

model = RNNModel()

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=15000,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

# Create trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=traindata,
    eval_dataset=testdata, 
    compute_metrics=compute_metrics,# training dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

# Train the model
# print(trainer.evaluate())

trainer.train()

print(trainer.evaluate())

  2%|▏         | 454/23500 [00:01<01:21, 284.24it/s]



                                           
                                                    
  2%|▏         | 511/23500 [00:02<01:55, 198.62it/s]

{'eval_loss': 0.604365885257721, 'eval_accuracy': 0.6723494938732019, 'eval_f1': 0.6332737030411449, 'eval_auc': 0.6710493444436116, 'eval_precision': 0.7070572569906791, 'eval_recall': 0.5734341252699784, 'eval_runtime': 0.1557, 'eval_samples_per_second': 12058.45, 'eval_steps_per_second': 758.07, 'epoch': 1.0}


  4%|▍         | 916/23500 [00:03<01:19, 285.07it/s]



                                           
                                                    
  4%|▍         | 973/23500 [00:03<01:53, 199.33it/s]

{'eval_loss': 0.5453975796699524, 'eval_accuracy': 0.7187000532765051, 'eval_f1': 0.7121046892039259, 'eval_auc': 0.7185223920256727, 'eval_precision': 0.7191629955947136, 'eval_recall': 0.7051835853131749, 'eval_runtime': 0.1583, 'eval_samples_per_second': 11859.757, 'eval_steps_per_second': 745.579, 'epoch': 2.0}


  6%|▌         | 1396/23500 [00:05<01:16, 289.32it/s]



                                           
                                                     
  6%|▌         | 1425/23500 [00:05<02:01, 181.77it/s]

...

{'eval_loss': 0.4860065281391144, 'eval_accuracy': 0.8066062866275973, 'eval_f1': 0.8034650785056848, 'eval_auc': 0.8065364865447989, 'eval_precision': 0.8056460369163952, 'eval_recall': 0.8012958963282938, 'eval_runtime': 0.1571, 'eval_samples_per_second': 11944.073, 'eval_steps_per_second': 750.879, 'epoch': 24.0}


 50%|████▉     | 11748/23500 [00:46<00:41, 285.81it/s]



                                           
                                                      
 50%|█████     | 11750/23500 [00:46<00:41, 285.81it/s]

 50%|█████     | 11750/23500 [00:46<00:46, 253.63it/s]


{'eval_loss': 0.48666319251060486, 'eval_accuracy': 0.8050079914757592, 'eval_f1': 0.8042780748663102, 'eval_auc': 0.8051011439589565, 'eval_precision': 0.7966101694915254, 'eval_recall': 0.8120950323974082, 'eval_runtime': 0.1557, 'eval_samples_per_second': 12056.825, 'eval_steps_per_second': 757.968, 'epoch': 25.0}
{'train_runtime': 46.3268, 'train_samples_per_second': 8100.065, 'train_steps_per_second': 507.266, 'train_loss': 0.3763123753324468, 'epoch': 25.0}


100%|██████████| 118/118 [00:00<00:00, 719.10it/s]

{'eval_loss': 0.4454619586467743, 'eval_accuracy': 0.8092701118806607, 'eval_f1': 0.8017718715393134, 'eval_auc': 0.8089097982571489, 'eval_precision': 0.8227272727272728, 'eval_recall': 0.7818574514038877, 'eval_runtime': 0.1656, 'eval_samples_per_second': 11334.586, 'eval_steps_per_second': 712.563, 'epoch': 25.0}


In [ ]:
# first LSTM
# 'eval_loss': 0.4162973463535309, 'eval_accuracy': 0.8286620835536753, 'eval_f1': 0.8205980066445182
